# Daten zu Feiertagen

### Feiertage von 2019 bis 2023 über API abrufen
- über API: https://date.nager.at/
- API URL: https://date.nager.at/api/v3/PublicHolidays/{year}/DE
- Filterung für Berlin (DE-BE)

In [41]:
import requests
import pandas as pd

# für Berlin
def get_feiertage(year):
    # URL für die Feiertage eines bestimmten Jahres in Deutschland
    url = f"https://date.nager.at/api/v3/PublicHolidays/{year}/DE"

    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        berlin_feiertage = [
            {"Datum": entry["date"], "Feiertag": entry["localName"]} 
            for entry in data 
            if entry["counties"] is None or "DE-BE" in entry["counties"]
        ]
        return berlin_feiertage
    else:
        print(f"Fehler beim Abrufen der Feiertage für {year}")
        return []

# alle von 2019 bis 2023 
feiertage_list = []
for year in range(2019, 2024):
    feiertage_list.extend(get_feiertage(year))

# in df umwandeln
df_feiertage = pd.DataFrame(feiertage_list)

# Datum in Format datetime umwandeln / ISO-Format
df_feiertage["Datum"] = pd.to_datetime(df_feiertage["Datum"])

# als csv speichern
# df_feiertage.to_csv("feiertage_berlin_2019_2023.csv", index=False)

### Datensatz Überblick

In [167]:
import pandas as pd

df_feiertage = pd.read_csv("daten/feiertage_berlin_2019_2023.csv")

In [168]:
df_feiertage.head(3)

,Datum,Feiertag
0,2019-01-01,Neujahr
1,2019-03-08,Internationaler Frauentag
2,2019-04-19,Karfreitag


In [169]:
df_feiertage.tail(3)

,Datum,Feiertag
48,2023-10-03,Tag der Deutschen Einheit
49,2023-12-25,Erster Weihnachtstag
50,2023-12-26,Zweiter Weihnachtstag


In [170]:
# Überblick
df_feiertage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Datum     51 non-null     object
 1   Feiertag  51 non-null     object
dtypes: object(2)
memory usage: 948.0+ bytes


In [171]:
# Statistische Zusammenfassung (für numerische Spalten)
df_feiertage.describe()

,Datum,Feiertag
count,51,51
unique,51,11
top,2019-01-01,Neujahr
freq,1,5


### Fehlende Werte

In [172]:
df_feiertage.isnull().sum()

Datum       0
Feiertag    0
dtype: int64

In [173]:
df_feiertage['Datum'] = pd.to_datetime(df_feiertage['Datum'])

In [174]:
df_feiertage.head()

,Datum,Feiertag
0,2019-01-01,Neujahr
1,2019-03-08,Internationaler Frauentag
2,2019-04-19,Karfreitag
3,2019-04-22,Ostermontag
4,2019-05-01,Tag der Arbeit


## Feiertage als Dummy-Variablen

In [175]:
# Sicherstellen, dass das Datum als Spalte beibehalten wird
df_feiertage = df_feiertage[['Datum', 'Feiertag']]  # Kopiere nur die Spalten, die du brauchst

# Erstelle Dummy-Variablen für die 'Feiertag'-Spalte
df_feiertage_dummies = pd.get_dummies(df_feiertage['Feiertag'], prefix='Feiertag')

# Umwandeln von True/False in 1/0
df_feiertage_dummies = df_feiertage_dummies.astype(int)

# Kombiniere die Dummy-Variablen mit dem ursprünglichen DataFrame
df_feiertage = pd.concat([df_feiertage, df_feiertage_dummies], axis=1)

# Jetzt ist das 'Datum' im DataFrame und es werden nur die Dummy-Variablen für Feiertage hinzugefügt.
df_feiertage.head()


,Datum,Feiertag,Feiertag_Christi Himmelfahrt,Feiertag_Erster Weihnachtstag,Feiertag_Internationaler Frauentag,Feiertag_Karfreitag,Feiertag_Neujahr,Feiertag_Ostermontag,Feiertag_Pfingstmontag,Feiertag_Tag der Arbeit,Feiertag_Tag der Befreiung,Feiertag_Tag der Deutschen Einheit,Feiertag_Zweiter Weihnachtstag
0,2019-01-01,Neujahr,0,0,0,0,1,0,0,0,0,0,0
1,2019-03-08,Internationaler Frauentag,0,0,1,0,0,0,0,0,0,0,0
2,2019-04-19,Karfreitag,0,0,0,1,0,0,0,0,0,0,0
3,2019-04-22,Ostermontag,0,0,0,0,0,1,0,0,0,0,0
4,2019-05-01,Tag der Arbeit,0,0,0,0,0,0,0,1,0,0,0


In [176]:
df_feiertage.head()

,Datum,Feiertag,Feiertag_Christi Himmelfahrt,Feiertag_Erster Weihnachtstag,Feiertag_Internationaler Frauentag,Feiertag_Karfreitag,Feiertag_Neujahr,Feiertag_Ostermontag,Feiertag_Pfingstmontag,Feiertag_Tag der Arbeit,Feiertag_Tag der Befreiung,Feiertag_Tag der Deutschen Einheit,Feiertag_Zweiter Weihnachtstag
0,2019-01-01,Neujahr,0,0,0,0,1,0,0,0,0,0,0
1,2019-03-08,Internationaler Frauentag,0,0,1,0,0,0,0,0,0,0,0
2,2019-04-19,Karfreitag,0,0,0,1,0,0,0,0,0,0,0
3,2019-04-22,Ostermontag,0,0,0,0,0,1,0,0,0,0,0
4,2019-05-01,Tag der Arbeit,0,0,0,0,0,0,0,1,0,0,0


In [177]:
df_feiertage = df_feiertage.drop(columns=["Feiertag"])

In [178]:
df_feiertage.head()

,Datum,Feiertag_Christi Himmelfahrt,Feiertag_Erster Weihnachtstag,Feiertag_Internationaler Frauentag,Feiertag_Karfreitag,Feiertag_Neujahr,Feiertag_Ostermontag,Feiertag_Pfingstmontag,Feiertag_Tag der Arbeit,Feiertag_Tag der Befreiung,Feiertag_Tag der Deutschen Einheit,Feiertag_Zweiter Weihnachtstag
0,2019-01-01,0,0,0,0,1,0,0,0,0,0,0
1,2019-03-08,0,0,1,0,0,0,0,0,0,0,0
2,2019-04-19,0,0,0,1,0,0,0,0,0,0,0
3,2019-04-22,0,0,0,0,0,1,0,0,0,0,0
4,2019-05-01,0,0,0,0,0,0,0,1,0,0,0


In [179]:
# Datensatz speichern
df_feiertage.to_csv("feiertage_berlin_dummies.csv", index=False)

### Mappen mit allen Tagen

In [152]:
# Erstellen eines DataFrames mit allen Daten von 2019 bis Ende 2023
date_range = pd.date_range(start="2019-01-01", end="2023-12-31", freq="D")
df_all_dates = pd.DataFrame(date_range, columns=["Datum"])

In [153]:
df_all_dates["Datum"] = pd.to_datetime(df_all_dates["Datum"])

In [154]:
df_all_dates.head(3)

,Datum
0,2019-01-01
1,2019-01-02
2,2019-01-03


In [155]:
df_feiertage["Datum"] = pd.to_datetime(df_feiertage["Datum"])

In [156]:
df_feiertage.head(3)

,Datum,Feiertag,Feiertag_Christi Himmelfahrt,Feiertag_Erster Weihnachtstag,Feiertag_Internationaler Frauentag,Feiertag_Karfreitag,Feiertag_Neujahr,Feiertag_Ostermontag,Feiertag_Pfingstmontag,Feiertag_Tag der Arbeit,Feiertag_Tag der Befreiung,Feiertag_Tag der Deutschen Einheit,Feiertag_Zweiter Weihnachtstag
0,2019-01-01,Neujahr,0,0,0,0,1,0,0,0,0,0,0
1,2019-03-08,Internationaler Frauentag,0,0,1,0,0,0,0,0,0,0,0
2,2019-04-19,Karfreitag,0,0,0,1,0,0,0,0,0,0,0


In [157]:
# Spalte Feiertag entfernen
df_feiertage = df_feiertage.drop(columns=["Feiertag"])

In [158]:
df_feiertage.head()

,Datum,Feiertag_Christi Himmelfahrt,Feiertag_Erster Weihnachtstag,Feiertag_Internationaler Frauentag,Feiertag_Karfreitag,Feiertag_Neujahr,Feiertag_Ostermontag,Feiertag_Pfingstmontag,Feiertag_Tag der Arbeit,Feiertag_Tag der Befreiung,Feiertag_Tag der Deutschen Einheit,Feiertag_Zweiter Weihnachtstag
0,2019-01-01,0,0,0,0,1,0,0,0,0,0,0
1,2019-03-08,0,0,1,0,0,0,0,0,0,0,0
2,2019-04-19,0,0,0,1,0,0,0,0,0,0,0
3,2019-04-22,0,0,0,0,0,1,0,0,0,0,0
4,2019-05-01,0,0,0,0,0,0,0,1,0,0,0


In [159]:
merged_df = pd.merge(df_all_dates, df_feiertage, on="Datum", how="left")

In [160]:
merged_df.head()

,Datum,Feiertag_Christi Himmelfahrt,Feiertag_Erster Weihnachtstag,Feiertag_Internationaler Frauentag,Feiertag_Karfreitag,Feiertag_Neujahr,Feiertag_Ostermontag,Feiertag_Pfingstmontag,Feiertag_Tag der Arbeit,Feiertag_Tag der Befreiung,Feiertag_Tag der Deutschen Einheit,Feiertag_Zweiter Weihnachtstag
0,2019-01-01,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
merged_df.fillna(False, inplace=True)

In [166]:
merged_df.head()

,Datum,Feiertag_Christi Himmelfahrt,Feiertag_Erster Weihnachtstag,Feiertag_Internationaler Frauentag,Feiertag_Karfreitag,Feiertag_Neujahr,Feiertag_Ostermontag,Feiertag_Pfingstmontag,Feiertag_Tag der Arbeit,Feiertag_Tag der Befreiung,Feiertag_Tag der Deutschen Einheit,Feiertag_Zweiter Weihnachtstag
0,2019-01-01,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [163]:
# Umwandeln der Dummy-Variablen in Integer (0 und 1)
merged_df.iloc[:, 1:] = merged_df.iloc[:, 1:].astype(int)

In [164]:
merged_df.head()

,Datum,Feiertag_Christi Himmelfahrt,Feiertag_Erster Weihnachtstag,Feiertag_Internationaler Frauentag,Feiertag_Karfreitag,Feiertag_Neujahr,Feiertag_Ostermontag,Feiertag_Pfingstmontag,Feiertag_Tag der Arbeit,Feiertag_Tag der Befreiung,Feiertag_Tag der Deutschen Einheit,Feiertag_Zweiter Weihnachtstag
0,2019-01-01,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
